# Cálculo numérico

En este capítulo presentamos brevemente algunas herramientas de cálculo numérico disponibles en [scipy](https://www.scipy.org/).

## Raíces de polinomios

En el tema dedicado a los *arrays* vimos algunas funciones de álgebra lineal numérica, entre las que destaca el solucionador de sistemas de ecuaciones lineales.

Las ecuaciones no lineales son en general más complicadas de resolver porque normalmente requieren aproximaciones sucesivas. Sin embargo, las raíces de polinomios son un caso especial que puede resolverse [de forma directa](https://en.wikipedia.org/wiki/Companion_matrix). Para ello se utiliza la función `roots`, que calcula las raíces de un polinomio expresado mediante la lista de sus coeficientes en orden decreciente.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Por ejemplo, para resolver la ecuación

$$x^4=16$$

la expresamos como un polinomio $p(x) = x^4- 16 = (1,0,0,0,-16) \cdot (x^4, x^3, x^2, x^1, x^0) = 0$.

In [ ]:
np.roots([1,0,0,0,-16])

Salvo ruido numérico, las soluciones obtenidas son $\pm2, \pm2i$.

Otro ejemplo: resuelve

$$ x^3 -2 x^2 + 5x + 6 = 0$$

In [ ]:
np.roots([1,-2,-5,6])

## Solución numérica de ecuaciones no lineales

Para resolver una ecuación cualquiera $f(x)=0$ tenemos que definir la función $f$ en Python de la forma usual y llamar a `fsolve` con un punto de partida.

In [ ]:
from scipy.optimize import fsolve

Resuelve

$$sin(x)+cos(2x)=0$$

In [ ]:
def f(x):
    return np.sin(x) + np.cos(2*x)

fsolve(f, 0)

Para hacer pruebas rápidas podemos definir sobre la marcha una función anónima.

In [ ]:
fsolve(lambda x: np.sin(x) + np.cos(2*x), 0)

Otro ejemplo: resuelve la ecuación de Kepler $x - b \sin(x)=a$ para $b=0.3$ y $a=0.4$.

In [ ]:
a = 0.4
b = 0.3

def f(x):
    return x - b*np.sin(x) - a

fsolve(f,0.2)

Las funciones matemáticas se pueden importar de `math`, de `scipy` o de `numpy`.

El resolvedor `fsolve` admite sistemas de ecuaciones no lineales $\vec f(\vec x)=\vec 0$. El proceso es el mismo: se define la función vectorial $f$ que describe el sistema de ecuaciones y se da un punto de partida para las incógnitas.

Resuelve

$$
\begin{align*}
x^2 - 3y &= 10\\
sin(x)+y &= 5
\end{align*}
$$

In [ ]:
def fun(z):
    x,y = z
    return [ x**2 - 3*y - 10
           , np.sin(x) + y - 5]

fsolve(fun,[0.1,-0.1])

Las ecuaciones no lineales pueden tener varias soluciones. La que encuentra `fsolve` depende del punto de partida.  

## Minimización

Un problema muy común es encontrar el mínimo de una cierta función de varias variables:
$\DeclareMathOperator*{\argmin}{argmin}
\argmin_{\vec x}f(\vec x)$. Para ello usamos `minimize`, que admite la función a minimizar y un punto de partida.

In [ ]:
from scipy.optimize import minimize

Encuentra $(x,y)$ que minimiza $(x-1)^2 + (y-2)^2-x+3y$

In [ ]:
def fun(z):
    x,y = z
    return (x-1)**2 + (y-2)**2 - x + 3*y

minimize(fun,[0.1,-0.1])

El resultado es un diccionario con información útil sobre el proceso de optimización.

Como este problema es solo de dos variables podemos ver en 3D la función y la posición del mínimo.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

r = np.linspace(0,2,30)
x,y = np.meshgrid(r,r)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

z = fun((x,y))

ax.plot_surface(x,y,z, cmap=cm.coolwarm, linewidth=0.5, rstride=2, cstride=2);

r = minimize(fun,[0.1,-0.1])

xo,yo  = r['x']

ax.plot3D([xo,xo],[yo,yo],[0,fun((xo,yo))],'.-');

La función anterior es cuadrática y tiene un único mínimo local. Si la función tiene varios mínimos locales la solución dependerá del punto de partida:

In [ ]:
def fun(z):
    x,y = z
    return np.cos(y)/(1+x**2)

minimize(fun,[0.1,0.1])

El mínimo encontrado es $(0,\pi)$, pero si cambiamos ligeramente el punto de partida encontramos otro:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

r = np.linspace(-5,5,100)
x,y = np.meshgrid(r,r)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

z = fun((x,y))

ax.plot_surface(x,y,z, cmap=cm.coolwarm, linewidth=0.5, rstride=2, cstride=2);

r = minimize(fun,[0.1,-0.1])

xo,yo  = r['x']

ax.plot3D([xo,xo],[yo,yo],[0,fun((xo,yo))],'.-');
ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('f(x,y)');

## Derivación numérica

A veces deseamos calcular la derivada de una función cuya expresión analítica se desconoce o es muy complicada, pero que podemos evaluar porque la tenemos en forma de código. Una posibilidad es calcular una aproximación numérica. Para ello usaremos el paquete 
[numdifftools](https://media.readthedocs.org/pdf/numdifftools/latest/numdifftools.pdf).

In [ ]:
# !pip install numdifftools

import sys
if 'pyodide' in sys.modules:
    %pip install numdifftools

Veamos un ejemplo. Calcula una aproximación numérica para $f'(2)$ cuando 

$$f(x) = \sin(2x)*\exp(\cos(x))$$

In [ ]:
from numpy import sin,cos, exp

def f(x):
    return np.sin(2*x)*np.exp(np.cos(x))

El valor real es:

In [ ]:
(lambda x: (-np.sin(x)*np.sin(2*x) + 2*np.cos(2*x))*np.exp(np.cos(x)))(2)

In [ ]:
import numdifftools as nd

df = nd.Derivative(f)

df(2)

In [ ]:
df = nd.Derivative(f, full_output=True, order=4)
df(2)

## Integración numérica

Para calcular integrales definidas usamos `quad`.

In [ ]:
from scipy.integrate import quad

Calcula una aproximación numérica a la integral definida

$$\int_0^1 \frac{4}{1+x^2}dx$$

In [ ]:
quad(lambda x: 4/(1+x**2),0,1)

Devuelve el resultado y el error estimado.

## Ecuaciones diferenciales

Hemos dedicado otro tema a la solución numérica de ecuaciones diferenciales. Aquí simplemente recordamos la forma de usar `odeint` con el ejemplo de un oscilador armónico con amortiguamiento.

Resuelve

$$\ddot{x}+0.95x+0.1\dot{x}=0$$

para $x(0)=10$, $\dot{x}(0)=0, t\in[0,20]$

In [ ]:
from scipy.integrate import odeint

def zdot(z,t):
    x,v = z
    return [v,-0.95*x-0.1*v]

t = np.linspace(0,20,1000)
x,v = odeint(zdot,[10,0],t).T
plt.plot(t,x,t,v);

In [ ]:
plt.plot(x,v);
plt.axis('equal');